In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [4]:
from tensorflow.keras.datasets import mnist

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [7]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [8]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [9]:
model = Sequential()

In [10]:
model.add(Conv2D(8, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [11]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [12]:
model.add(Conv2D(16, (3,3), padding='same',
                activation='relu'))

In [13]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(20, activation='relu'))

In [19]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0

#### 組裝

In [21]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

### 4. step 2: fit

In [22]:
model.fit(x_train, y_train, batch_size=128, epochs=12)

Epoch 1/12
469/469 [==============================] - 31s 66ms/step - loss: 0.0899 - accuracy: 0.1378
Epoch 2/12
469/469 [==============================] - 31s 66ms/step - loss: 0.0893 - accuracy: 0.2005
Epoch 3/12
469/469 [==============================] - 31s 67ms/step - loss: 0.0878 - accuracy: 0.3196
Epoch 4/12
469/469 [==============================] - 32s 67ms/step - loss: 0.0761 - accuracy: 0.4580
Epoch 5/12
469/469 [==============================] - 31s 67ms/step - loss: 0.0387 - accuracy: 0.7404
Epoch 6/12
469/469 [==============================] - 31s 67ms/step - loss: 0.0266 - accuracy: 0.8195
Epoch 7/12
469/469 [==============================] - 36s 76ms/step - loss: 0.0232 - accuracy: 0.8416
Epoch 8/12
469/469 [==============================] - 31s 66ms/step - loss: 0.0213 - accuracy: 0.8538
Epoch 9/12
469/469 [==============================] - 31s 67ms/step - loss: 0.0202 - accuracy: 0.8598
Epoch 10/12
469/469 [==============================] - 31s 67ms/step - loss: 0.019

### Step 3. 預測

In [23]:
result = model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### 看看測試資料表現如何

In [24]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0179 - accuracy: 0.8749


In [25]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 87.49%


In [26]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [27]:
from ipywidgets import interact_manual

In [28]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [29]:
model.save('myCNNmodel.h5')